In [1]:
import ee
import geopandas as gpd
import geemap
import geersd
import pandas as pd

from pph import opca

In [2]:
ee.Initialize()

In [3]:
gdf_roi = gpd.read_file("../../scratch/moose_mountain_FT_OPC_analysis_AOI_7year_L8.shp")
gdf_roi.set_crs(4326, inplace=True)
aoi = ee.FeatureCollection(gdf_roi.__geo_interface__).geometry()

In [4]:
def fetch_gdf(t1, t2, clouds: float = 60):
    ic = geersd.Landsat8SR().filterBounds(aoi).filterDate(t1, t2)
    
    if clouds > 0:
        ic = ic.filterClouds(clouds)

    ic = ic.map(lambda x: x.set('date', x.date().format('YYYY-MM-dd')))

    gdf = ic.toGeoPandasDataFrame()
    
    column_names = [
        'date',
        'system:id',
        'system:time_end',
        'system:time_start',
        'CLOUD_COVER',
        'UTM_ZONE', 
        'WRS_PATH',
        'WRS_ROW', 
        'WRS_TYPE',
        'geometry',
    ]
    
    gdfout = gdf[column_names]
    gdfout.set_crs(4326, inplace=True)
    gdfout[['Year', 'Month', 'Day']] = gdfout['date'].str.split('-', expand=True)
    gdfout[['Year', 'Month', 'Day']] = gdfout[['Year', 'Month', 'Day']].astype(int)
    return gdfout

In [5]:
CLOUD = 100
gdfs = []

for i in range(2017, 2024):
    start, end = f'{i}-04-01', f"{i}-10-31"
    gdfs.append(fetch_gdf(start, end, CLOUD))

gdfs = pd.concat(gdfs)
# Path, Row
# 34, 25
gdfs = gdfs[(gdfs.WRS_PATH == 34) & (gdfs.WRS_ROW == 25)]

C:\Users\rhamilton\AppData\Local\miniconda3\envs\pph\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\rhamilton\AppData\Local\miniconda3\envs\pph\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\rhamilton\AppData\Local\miniconda3\envs\pph\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to b

In [6]:
gdfs.to_file(f"../../scratch/moose_mnt_l8_34_25_{CLOUD}.geojson", driver="GeoJSON")